# Tutorial (1): How to run quket

Here we show how to run Quket using an example of UCCSD VQE applied to N$_2$/STO-6G.

[(1) Preparation](#(1)-Preparation)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(1.1) `create()` to prepare the simulation](#(1.1)-Use-create()-to-prepare-the-simulation)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(1.2) `print_mo_energy()` to print out orbital energies](#(1.2)-Use-print_mo_energy()-to-find-out-what-orbitals-are-actually-used-in-the-simulation)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(1.3) `fci2qubit()` to get FCI state(s)](#(1.3)-Use-fci2qubit()-to-get-FCI-state(s)-in-the-qubit-representation)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(1.4) Pauli list for variational circuit](#(1.4)-Pauli-list-for-variational-circuit)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(1.5) `print_state()` to check the quantum states](#(1.5)-print_state()-to-check-the-quantum-states)  

[(2) Calculation](#(2)-Calculation)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(2.1) `run()` to perform calculation](#(2.1)-Use-run()-to-perform-calculation)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(2.2) Converged variational parameters](#(2.2)-Converged-variational-parameters)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(2.3) `fidelity()` to check the converged UCCSD state](#(2.3)-fidelity()-to-check-the-converged-UCCSD-state)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(2.4) `vqe()` is an alternative to `run()` for VQE](#(2.4)-vqe()-is-an-alternative-to-run()-for-VQE)  

[(3) Further functionalities of `QuketData`](#(3)-Further-functionalities-of-QuketData)  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(3.1) `print_state()`](#(3.1)-print_state())  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(3.2) `get_E()`](#(3.2)-get_E())  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(3.3) `get_S2()` and `get_N()`](#(3.3)-get_S2()-and-get_N())  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(3.4) `get_1RDM()`](#(3.4)-get_1RDM())  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(3.5) `save()`, `load()`, `copy()`](#(3.5)-save(),-load(),-copy())  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;[(3.6) `set()` and `initialize()`](#(3.6)-set()-and-initialize())





## (1) Preparation

In [1]:
# import necessary modules
from quket import *
from quket.utils import *
import quket.config as cf


mpi4py is not imported. no MPI.


### (1.1) Use `create()` to prepare the simulation
For molecular systems, important arguments are  
`basis`: Basis set (e.g., sto-6g)  
`n_electrons`: Number of active electrons to be considered in the simulation (e.g., 6)  
`n_orbitals`: Number of active orbitals to be mapped onto qubits. Orbital indices are automatically determined based on Hartree Fock configuration and `n_electrons`.  
`geometry`: Molecular geometry can be set either by PySCF format, cartesian format, or z-matrix format,
```python
    geometry = [['N', (0, 0, 0)], ['N', (0, 0, 1.098)]]
    geometry = "N 0 0 0\n N 0 0 1.098" 
    geometry = "N; N 1 1.098"
```
where `;` and `\n` are used to indicate a break.  

For VQE and other ansatz-based algorithms, `ansatz` is required. Here, we use `sauccsd` (spin-adapted uccsd). The list of currently available ansatze is found by 
```python
cf.vqe_ansatz_list
```
Methods available in the currrent Quket is also found by
```python
cf.method_list
```
for which the default is VQE, so we can skip specifying `method` in the following example.

In [2]:
### Create QuketData for N2 ###
Q = create(basis="sto-6g", 
                 ansatz="sauccsd", 
                 n_orbitals =6, 
                 n_electrons=6, 
                 geometry = "N; N 1 1.098", 
                 mapping = 'bk',
                )

Basis set = sto-6g

*** Geometry ******************************
  N     0.0000000    0.0000000    0.0000000
  N     1.0980000    0.0000000    0.0000000
*******************************************

Symmetry Dooh : D2h(Abelian)
E[FCI]    = -108.669172966971     (Spin = 1   Ms = 0)
E[HF]     = -108.541914960860     (Spin = 1   Ms = 0)


Overwritten attributes  contract_2e  of <class 'pyscf.fci.direct_spin1_symm.FCISolver'>


Tapering-Off Results:
List of redundant qubits:  [0, 1, 3, 5, 7]
Qubit: 0    Tau: 1.0 [Z0 Z2 Z4 Z6 Z8 Z10]
Qubit: 1    Tau: 1.0 [Z1 Z9]
Qubit: 3    Tau: 1.0 [Z3 Z9 Z11]
Qubit: 5    Tau: 1.0 [Z5 Z9 Z11]
Qubit: 7    Tau: 1.0 [Z7 Z11]

Symmetry-forbidden pauli operators are removed.
NBasis = 10


#### If necessary arguments are provided, `create()` sets everything up in order to perform the calculation.
A brief summary can be printed.

In [3]:
print(Q)


*** Geometry ******************************
  N     0.0000000    0.0000000    0.0000000
  N     1.0980000    0.0000000    0.0000000
*******************************************

Basis        = sto-6g
NBasis       = 10
Ne           = 6
Norbs        = 6
Multiplicity = 1
Method       = vqe
Ansatz       = sauccsd
Mapping      = bravyi_kitaev
Nqubits      = 12
Energy       = 0.0
Converged    = False
Derivative   = Analytical (Exact)



`create()` also generates `pauli_list` and `theta_list`.  
`pauli_list`: List of `QubitOperator`s that will be used to form a quantum circuit, parameterized by `theta_list`, which is initially zero.   
The length of `pauli_list` should match that of `theta_list`. 

In [4]:
len(Q.pauli_list) == len(Q.theta_list)

True

If `symmetry = True`, then `pauli_list` removes the symmetry-forbidden pauli operators.

### (1.2) Use `print_mo_energy()` to find out what orbitals are actually used in the simulation

In [5]:
Q.print_mo_energy()


[Molecular Orbitals]
---+-------+------------+------------------
 # |  Sym  |   energy   |     category
---+-------+------------+------------------
 0 |   Ag  |   -15.7220 | Frozen Core
 1 |  B1u  |   -15.7197 | Frozen Core
 2 |   Ag  |    -1.4497 | Frozen Core
 3 |  B1u  |    -0.7317 | Frozen Core
 4 |  B2u  |    -0.5775 | Active (occupied)
 5 |  B3u  |    -0.5775 | Active (occupied)
 6 |   Ag  |    -0.5441 | Active (occupied)
 7 |  B2g  |     0.2707 | Active (virtual)
 8 |  B3g  |     0.2707 | Active (virtual)
 9 |  B1u  |     1.0924 | Active (virtual)
---+-------+------------+------------------



### (1.3) Use `fci2qubit()` to get FCI state(s) in the qubit representation
Note this is slow compared to the sophisticated FCI algorithm implemented in PySCF.  
Setting `nroots` in `create(nroots=***)` can yield excited states.

In [6]:
Q.fci2qubit()

Davidson convergence achieved.
FCI in Qubits
(FCI state : E = -108.6691729679  multiplicity = 1.0)
      Basis              Coef
| 000000010101 > : +0.9614 +0.0000i
| 000001010001 > : -0.1383 +0.0000i
| 000100010100 > : -0.1383 +0.0000i



#### Initial states of `fci2qubit()` are usually the determinants with the lowest energy (in the same number and Sz symmetry sector as HF).
This can be modified by setting `init_states`.
`init_states` has a similar structure to `multi`, which is explained in Tutorial-5: it is a list of strings of single or multi determinants.

For example,

```python
["|00111111>", "|11001111>"]
```
provides the HF determinant and a HOMO-LUMO doubly-excited determinant as initial guesses.

These determinants are in the Jordan-Wigner representation (if `QuketData` employes Bravyi-Kitaev mapping and/or reduced mapping by tapering-off scheme, `fci2qubit` automatically performs the necessary transformations).

In [7]:
Q.fci2qubit(init_states = ["|0111111>", "|11001111>"], verbose=True)

2022-09-16 00:31:07.661124:  Entered fci2qubit
Target spin s = 1
Diagonal matrix elements prepared.
Cycle  State       Energy      Grad
[ 0]      0:  -108.5438824039   4.82e-01  
          1:  -107.8064134385   2.10e-01  
[ 1]      0:  -108.6539989143   1.65e-01  
          1:  -107.8280983815   1.89e-01  
[ 2]      0:  -108.6665276765   5.93e-02  
          1:  -107.8694723847   6.09e-02  
[ 3]      0:  -108.6689203411   2.35e-02  
          1:  -107.8744294490   3.24e-02  
[ 4]      0:  -108.6690852711   1.47e-02  
          1:  -107.8751393711   1.37e-02  
[ 5]      0:  -108.6691457777   7.25e-03  
          1:  -107.8753601964   7.53e-03  
[ 6]      0:  -108.6691699816   1.98e-03  
          1:  -107.8753905846   4.59e-03  
[ 7]      0:  -108.6691722032   1.44e-03  
          1:  -107.8753961078   3.03e-03  
[ 8]      0:  -108.6691728221   7.90e-04  
          1:  -107.9297590328   3.59e-01  
[ 9]      0:  -108.6691729628   1.15e-04  
          1:  -107.9663135707   5.52e-02  
[10]

#### Caution
Note that for chemical Hamiltonians (`Q.model == 'chemical'`), `fci2qubit` implicitly confines the Hilbert space to the same symmetry sector as the underlying HF, that is, the same electron numbers for $\alpha$ and $\beta$. So, if `initial_states` contains the wrong symmetry state, `fci2qubit` can fail. For `Q.model == 'user-defined'`, the entire Hilbert space is treated and there is no problem.

### Tips:
Elements of `init_states` can be `QuantumState`.

`nroots` results are obtained regardless of the number of states specified in `init_states`.

In [8]:
print(type(Q.init_state))
Q.fci2qubit(nroots=4, init_states = [Q.init_state])

<class 'quket.lib.qulacs.QuantumState'>
Davidson convergence achieved.
FCI in Qubits
(FCI state : E = -108.6691729679  multiplicity = 1.0)
      Basis              Coef
| 000000010101 > : +0.9614 +0.0000i
| 000001010001 > : -0.1383 +0.0000i
| 000100010100 > : -0.1383 +0.0000i

(FCI state : E = -107.9680846467  multiplicity = 1.0)
      Basis              Coef
| 000001010001 > : +0.3612 +0.0000i
| 000001010100 > : -0.1365 +0.0000i
| 000100010001 > : -0.1365 +0.0000i
| 000100010100 > : +0.3612 +0.0000i
| 101010010111 > : -0.3802 +0.0000i
| 101011010110 > : +0.4389 +0.0000i
| 101110010011 > : +0.4389 +0.0000i
| 101111010010 > : -0.3802 +0.0000i

(FCI state : E = -107.8870133077  multiplicity = 1.0)
      Basis              Coef
| 000001010001 > : +0.3533 +0.0000i
| 000001010100 > : -0.3533 +0.0000i
| 000100010001 > : -0.3533 +0.0000i
| 000100010100 > : +0.3533 +0.0000i
| 101011010011 > : +0.3533 +0.0000i
| 101011010110 > : -0.3533 +0.0000i
| 101110010011 > : -0.3533 +0.0000i
| 10111001011

### (1.4) Pauli list for variational circuit
To see the corresponding pauli strings that are used to generate the variational circuit, you may check 
```python
    Q.pauli_list
```
For spin-adapted ansatze, some paulis $P_j, P_{j}', \cdots$ should have the same theta although these paulis. If they commute, we can simply add them to treat simultaneously, $P_j \leftarrow P_j+P_{j}'+\cdots$. However, if some of them do not commute with each other, the corresponding element of `pauli_list` is itself a list of paulis, [$P_j, P_j'$].

In [9]:
### 0th pauli is a sum of paulis (double excitation from 0,1 to 6,7) because they all commute
Q.pauli_list[0]

-0.25j [X0 Z1 Z3 Z5 Y6 Z7] +
-0.25j [X0 Z1 Y6] +
-0.25j [X0 Z3 Z5 Y6 Z7] +
-0.25j [X0 Y6] +
0.25j [Y0 Z1 Z3 Z5 X6 Z7] +
0.25j [Y0 Z1 X6] +
0.25j [Y0 Z3 Z5 X6 Z7] +
0.25j [Y0 X6]

In [10]:
### 3rd pauli is a list of paulis because some of them do not commute
Q.pauli_list[3]

[0.125j [X0 Y1 X2 Z3 Z5 X6 Y7 Y8 X9 X11] +
 0.125j [X0 Y1 X2 Z3 Z5 Y6 Y7 X8 X9 X11] +
 -0.125j [X0 Y1 Y2 Z3 Z5 X6 Y7 X8 X9 X11] +
 0.125j [X0 Y1 Y2 Z3 Z5 Y6 Y7 Y8 X9 X11] +
 -0.125j [Y0 Y1 X2 Z3 Z5 X6 Y7 X8 X9 X11] +
 0.125j [Y0 Y1 X2 Z3 Z5 Y6 Y7 Y8 X9 X11] +
 -0.125j [Y0 Y1 Y2 Z3 Z5 X6 Y7 Y8 X9 X11] +
 -0.125j [Y0 Y1 Y2 Z3 Z5 Y6 Y7 X8 X9 X11] +
 0.125j [Z0 X1 Z3 X7 Y9 X11] +
 0.125j [Z0 X1 Z5 Z6 Y7 Z8 X9 X11] +
 0.125j [Z0 Y1 Z2 Z3 Z5 Z6 Y7 Y9 X11] +
 -0.125j [Z0 Y1 Z2 X7 Z8 X9 X11] +
 0.125j [X1 Z2 Z3 Z5 Z6 Y7 Z8 X9 X11] +
 0.125j [X1 Z2 X7 Y9 X11] +
 -0.125j [Y1 Z3 X7 Z8 X9 X11] +
 0.125j [Y1 Z5 Z6 Y7 Y9 X11],
 0.125j [X0 X1 Z3 X7 Y8 X9 X11] +
 0.125j [X0 X1 Z5 Z6 Y7 X8 X9 X11] +
 0.125j [X0 Y1 Z2 Z3 Z5 Z6 Y7 Y8 X9 X11] +
 -0.125j [X0 Y1 Z2 X7 X8 X9 X11] +
 -0.125j [Y0 X1 Z3 X7 X8 X9 X11] +
 0.125j [Y0 X1 Z5 Z6 Y7 Y8 X9 X11] +
 -0.125j [Y0 Y1 Z2 Z3 Z5 Z6 Y7 X8 X9 X11] +
 -0.125j [Y0 Y1 Z2 X7 Y8 X9 X11] +
 0.125j [Z0 Y1 X2 Z3 Z5 X6 Y7 Y9 X11] +
 0.125j [Z0 Y1 X2 Z3 Z5 Y6 Y7 Z8 X9 X11

### (1.5) `print_state()` to check the quantum states
Attributes `init_state`, `state`, and `fci_states` contain the initial, current (final), and FCI states in the form of `qulacs`'s `QuantumState`.

Use `print_state()` to print them.

In [11]:
print_state(Q.init_state, name="initial HF state")
print_state(Q.state, name="current state")
print_state(Q.fci_states[0]['state'], name="fci exact state")

initial HF state
      Basis              Coef
| 000000010101 > : +1.0000 +0.0000i

current state
      Basis              Coef
| 000000010101 > : +1.0000 +0.0000i

fci exact state
      Basis              Coef
| 000000010101 > : +0.9614 +0.0000i
| 000001010001 > : -0.1383 +0.0000i
| 000100010100 > : -0.1383 +0.0000i



## (2) Calculation

### (2.1) Use `run()` to perform calculation 
It will run the method specified with the `method` attribute.  
If `fci2qubit()` has been performed, VQE runs `fidelity()` at each cycle to show the fidelity.

In [12]:
# Run UCCSD
Q.run()

Entered VQE driver
Performing VQE for sauccsd
Number of VQE parameters: 15
Initial configuration: |000000111111>
Convergence criteria: ftol = 1E-09, gtol = 1E-05
Derivatives: Analytical
Circuit order: Exp[T1] Exp[T2] |0>
Initial: E[sauccsd] = -108.541914960860  <S**2> = +0.000000  Fidelity = 0.924343  rho = 1  
      1: E[sauccsd] = -108.646352933729  <S**2> = +0.000001  Fidelity = 0.976129  Grad = 6.26e-01  CPU Time =    0.33491  (0.00 / step)
      2: E[sauccsd] = -108.659301110535  <S**2> = +0.000002  Fidelity = 0.987591  Grad = 3.20e-01  CPU Time =    0.12638  (0.00 / step)
      3: E[sauccsd] = -108.666470234984  <S**2> = +0.000003  Fidelity = 0.996816  Grad = 1.46e-01  CPU Time =    0.12579  (0.00 / step)
      4: E[sauccsd] = -108.668164908414  <S**2> = +0.000004  Fidelity = 0.999154  Grad = 7.34e-02  CPU Time =    0.12691  (0.00 / step)
      5: E[sauccsd] = -108.668556521107  <S**2> = +0.000007  Fidelity = 0.999681  Grad = 7.11e-02  CPU Time =    0.12529  (0.00 / step)
      6

### (2.2) Converged variational parameters
To see the optimized theta values, you may check
```python
    printmat(Q.theta_list)
```
Here, `printmat` is a subroutine that prints out a matrix/vector.

In [13]:
printmat(Q.theta_list)


               0          
    0     -0.0103013  
    1     -0.0723087  
    2     -0.0055647  
    3     -0.0885835  
    4      0.0026076  
    5     -0.0361066  
    6     -0.0720568  
    7     -0.0101847  
    8     -0.0056327  
    9      0.0026044  
   10     -0.0316131  
   11     -0.0316144  
   12     -0.0190303  
   13     -0.0190477  
   14     -0.0137579  



### (2.3) `fidelity()` to check the converged UCCSD state
Compare the UCCSD state and FCI states.

In [14]:
print_state(Q.state, name="final uccsd state")
print_state(Q.fci_states[0]['state'], name="fci exact state")
Q.fidelity(state=Q.state) # Default state = Q.state 

final uccsd state
      Basis              Coef
| 000000010101 > : +0.9613 +0.0000i
| 000001010001 > : -0.1398 +0.0000i
| 000100010100 > : -0.1398 +0.0000i

fci exact state
      Basis              Coef
| 000000010101 > : +0.9614 +0.0000i
| 000001010001 > : -0.1383 +0.0000i
| 000100010100 > : -0.1383 +0.0000i



0.9998389958678328

### (2.4) `vqe()` is an alternative to `run()` for VQE
`vqe()` simply runs VQE with `pauli_list` and `theta_list` (which is zeroed-out because `pauli_list` has been changed).  
It can be used even if `method != "vqe"`, but `theta_list` will be overwritten.

In [15]:
# Q.theta_list contains converged theta from `run()`, so initialize to zero
Q.theta_list *= 0
Q.vqe()

      1: E[sauccsd] = -108.646352933729  <S**2> = +0.000001  Fidelity = 0.976129  Grad = 6.26e-01  CPU Time =    0.31867  (0.00 / step)
      2: E[sauccsd] = -108.659301110535  <S**2> = +0.000002  Fidelity = 0.987591  Grad = 3.20e-01  CPU Time =    0.12655  (0.00 / step)
      3: E[sauccsd] = -108.666470234984  <S**2> = +0.000003  Fidelity = 0.996816  Grad = 1.46e-01  CPU Time =    0.12802  (0.00 / step)
      4: E[sauccsd] = -108.668164908414  <S**2> = +0.000004  Fidelity = 0.999154  Grad = 7.34e-02  CPU Time =    0.12834  (0.00 / step)
      5: E[sauccsd] = -108.668556521106  <S**2> = +0.000007  Fidelity = 0.999681  Grad = 7.11e-02  CPU Time =    0.13018  (0.00 / step)
      6: E[sauccsd] = -108.668735126514  <S**2> = +0.000009  Fidelity = 0.999814  Grad = 1.32e-02  CPU Time =    0.12758  (0.00 / step)
      7: E[sauccsd] = -108.668745328372  <S**2> = +0.000009  Fidelity = 0.999835  Grad = 4.44e-03  CPU Time =    0.12693  (0.00 / step)
      8: E[sauccsd] = -108.668746776280  <S**2> 

## (3) Further functionalities of `QuketData`

### (3.1) `print_state()`
This simply calls `print_state()` using `Q.state` as a default state.

In [16]:
Q.print_state()
Q.print_state(Q.init_state)

      Basis              Coef
| 000000010101 > : +0.9613 +0.0000i
| 000001010001 > : -0.1398 +0.0000i
| 000100010100 > : -0.1398 +0.0000i

      Basis              Coef
| 000000010101 > : +1.0000 +0.0000i



### (3.2) `get_E()`
This evaluates the expectation value of energy using `Q.state` as a default state.

In [17]:
# UCCSD energy (Q.state)
print("UCCSD final energy = ", Q.get_E())
# Hartree-Fock energy (init_state = HF)
print("HF initial energy = ", Q.get_E(Q.init_state))

UCCSD final energy =  -108.66874684645686
HF initial energy =  -108.54191496085956


### (3.3) `get_S2()` and `get_N()` 
They similarly evaluate the expectation value of $S^2$ and $N$ (using `Q.state` as a default state)

In [18]:
print("UCCSD <S**2> = ", Q.get_S2())
print("UCCSD <N> = ", Q.get_N())

UCCSD <S**2> =  9.050300602164718e-06
UCCSD <N> =  5.999999999999974


### (3.4) `get_1RDM()`
It evaluates the one-particle density matrix (alpha and beta spins).

In [19]:
### Compute UCCSD 1RDM
Q.get_1RDM()
### Now have DA, DB, RelDA (relaxed DA), etc.
# Can use printmat() 
printmat(Q.DA, "unrelaxed DA")
printmat(Q.RelDA, "relaxed DA")

 === Computing 1RDM === 
 === Computing 2RDM === 

unrelaxed DA

               0              1              2              3              4              5              6              7              8              9          

    0      1.0000000      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000  
    1      0.0000000      1.0000000      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000  
    2      0.0000000      0.0000000      1.0000000      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000  
    3      0.0000000      0.0000000      0.0000000      1.0000000      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000      0.0000000  
    4      0.0000000      0.0000000      0.0000000      0.0000000      0.9651076      0.0000000      0.0000000      0.0000000      0.0000000

### (3.5) `save()`, `load()`, `copy()`
Save and load the result in `QuketData`.  
`QuketData` can be also copied.

In [20]:
Q.save('q.qkt')

Saved in q.qkt.


In [21]:
X = create()
X.load('q.qkt')

Data loaded successfully.


In [22]:
print(Q.energy, X.energy)
X.print_state()

-108.66874684645687 -108.66874684645687
      Basis              Coef
| 000000010101 > : +0.9613 +0.0000i
| 000001010001 > : -0.1398 +0.0000i
| 000100010100 > : -0.1398 +0.0000i



In [23]:
# Successfuly loaded, but the memory addresses are different
print(X.pauli_list == Q.pauli_list)
print(id(X.pauli_list) == id(Q.pauli_list))

True
False


In [24]:
# Copy bypasses save and load.
X = Q.copy()

### (3.6) `set()` and `initialize()`
You can change the active space by using `set()` to change the parameters and `initialize()` to initialize Hamiltonian and such.  
For example, we can restrict the active space to contain only 4 electrons and $\pi$, $\pi^*$ orbitals.  
Since the active space is defined with the orbitals around HOMO and LUMO, 4th and 6th orbitals should be switched.  
This can be done by using `alter`.

In [25]:
Q.set(alter=[4,6])
# Redefine numbers of active orbitals and electrons, and initial state.
Q.set(n_orbitals=4, n_electrons=4, det="00001111")
Q.initialize()
Q.print_mo_energy()

Basis set = sto-6g

*** Geometry ******************************
  N     0.0000000    0.0000000    0.0000000
  N     1.0980000    0.0000000    0.0000000
*******************************************

Symmetry Dooh : D2h(Abelian)
E[FCI]    = -108.582282828832     (Spin = 1   Ms = 0)
E[HF]     = -108.541914960860     (Spin = 1   Ms = 0)
Orbital switched:
   4 <-> 6 
Existing theta_list is inconsistent :
   size of pauli_list 14 != size of theta_list 15.
Ovewrite theta_list by zero.
Tapering-Off Results:
List of redundant qubits:  [0, 1, 3, 7]
Qubit: 0    Tau: 1.0 [Z0 Z2 Z4 Z6]
Qubit: 1    Tau: 1.0 [Z1 Z5]
Qubit: 3    Tau: 1.0 [Z3]
Qubit: 7    Tau: 1.0 [Z7]

Symmetry-forbidden pauli operators are removed.

[Molecular Orbitals]
---+-------+------------+------------------
 # |  Sym  |   energy   |     category
---+-------+------------+------------------
 0 |   Ag  |   -15.7220 | Frozen Core
 1 |  B1u  |   -15.7197 | Frozen Core
 2 |   Ag  |    -1.4497 | Frozen Core
 3 |  B1u  |    -0.7317 | Fr

In [26]:
# Run vqe
Q.vqe()

      1: E[sauccsd] = -108.623150118523  <S**2> = +0.000037  Grad = 6.70e-01  CPU Time =    0.07685  (0.00 / step)
      2: E[sauccsd] = -108.638941665769  <S**2> = +0.000005  Grad = 2.42e-01  CPU Time =    0.02605  (0.00 / step)
      3: E[sauccsd] = -108.643018339026  <S**2> = +0.000002  Grad = 1.23e-01  CPU Time =    0.02568  (0.00 / step)
      4: E[sauccsd] = -108.644586262916  <S**2> = +0.000002  Grad = 6.25e-02  CPU Time =    0.02530  (0.00 / step)
      5: E[sauccsd] = -108.645011294982  <S**2> = +0.000003  Grad = 1.73e-02  CPU Time =    0.02684  (0.00 / step)
      6: E[sauccsd] = -108.645023794702  <S**2> = +0.000003  Grad = 5.50e-03  CPU Time =    0.02607  (0.00 / step)
      7: E[sauccsd] = -108.645025645439  <S**2> = +0.000003  Grad = 6.95e-04  CPU Time =    0.02613  (0.00 / step)
      8: E[sauccsd] = -108.645025673258  <S**2> = +0.000003  Grad = 1.23e-04  CPU Time =    0.02584  (0.00 / step)
  Final: E[sauccsd] = -108.645025673258  <S**2> = +0.000003  rho = 1 

(sauccsd 

### Currently, FCI energy is evaluated before performing `alter`.
So the FCI energy may be wrong. This should be fixed.  
Also, note that `fci2qubit()` exploits FCI coefficients provided by PySCF, so in this case it does not work.

### (3.7) Debug mode
Simply set the option `debug = True` or overwrite the parameter `debug` in the config file, `cf.debug = True`

In [27]:
Q.set(debug=True)
Q.initialize()
Q.run()

+----------------+
| CHEMICAL STATE |
+----------------+
	basis=sto-6g
	mo_basis=hf
	n_orbitals=10
	n_frozen_orbitals=5
	n_core_orbitals=0
	n_active_orbitals=4
	n_frozenv_orbitals=1
	n_secondary_orbitals=0
	n_electrons=14
	n_active_electrons=4
	excitation
	include={'c': 'a/s', 'a': 'a/s', 'cc': 'aa/as/ss', 'ca': 'aa/as/ss', 'aa': 'aa/as/ss'}
Basis set = sto-6g

*** Geometry ******************************
  N     0.0000000    0.0000000    0.0000000
  N     1.0980000    0.0000000    0.0000000
*******************************************

Symmetry Dooh : D2h(Abelian)
E[FCI]    = -108.582282828832     (Spin = 1   Ms = 0)
E[HF]     = -108.541914960860     (Spin = 1   Ms = 0)
Orbital switched:
   4 <-> 6 
Existing theta_list is inconsistent :
   size of pauli_list 14 != size of theta_list 6.
Ovewrite theta_list by zero.

Point Group Symmetry:
  SymmOP \ IRREP             B3u             B3u              Ag              Ag             B2g             B2g             B3g             B3g
       

In [28]:
print(cf.debug)

True
